In [21]:
%reload_ext autoreload
%autoreload 2

import json
import pandas as pd

from cobra.io import read_sbml_model
from cobra.io import write_sbml_model
from cobra.io.sbml import validate_sbml_model

from csm4cobra.consistency_analysis import UmFinder

from csm4cobra import reachability

from cobra.manipulation.modify import rename_genes

In [11]:
sbml_fname = 'Recon3DModel_301_no_isoforms.xml'
model_original = read_sbml_model(sbml_fname)

model = model_original.copy()

In [9]:
df_genes = pd.read_csv('./recon3d-store-genes.tsv', sep='\t')
df_genes = df_genes[['gene_number', 'symbol', 'gstart', 'gend', 'ensembl_gene']]

remove_isoforms_dict = {i:str(i).split('.')[0] for i in df_genes.gene_number}
df_genes = df_genes.replace(remove_isoforms_dict)
df_genes = df_genes.drop_duplicates()

df_genes = df_genes.set_index('gene_number')

In [26]:
gene_id_mappging = {g.id:df_genes.symbol[g.id] for g in model.genes}
rename_genes(model, gene_id_mappging)

In [30]:
#############################################
# Setting the correct boundary condition
#############################################

json_fname ="RPMI_1640_media.json"
with open(json_fname) as fh:
    rpmi_media = json.load(fh)

for r in rpmi_media:
    rpmi_media[r] *= -1

rpmi_media['EX_Tyr_ggn_e'] = 0

# Important: this sink "sink_tag_hs_c" should have a negative lower bound to predict growht 
rpmi_media['EX_glc_D_e'] = -5
rpmi_media['sink_tag_hs_c'] = -0.2

model.reactions.biomass_maintenance.objective_coefficient = 0
model.reactions.biomass_reaction.objective_coefficient = 1

model.reactions.ARTFR208.bounds = (0, 0)
model.reactions.ARTPLM3.bounds = (0, 0)

solution = model.optimize()
print("Optimal solution found",solution.objective_value)

for r in model.boundary:
    if r.lower_bound >= 0:
        continue
    r.lower_bound = 0

print("Skiping:", end=", ")
for r in rpmi_media:
    if r in model.reactions:
        rxn = model.reactions.get_by_id(r)
        rxn.lower_bound = rpmi_media[r]
    else:
        print(r, end=" ")

solution = model.optimize()
print("Optimal solution found",solution.objective_value)

Optimal solution found 0.0844431984381386
Skiping:, EX_cl_e EX_ca2_e EX_zn2_e EX_glc_e Optimal solution found 0.0844431984381386


In [31]:
json_fname ="All_biomass_compound.json"
with open(json_fname) as fh:
    target_products = json.load(fh)
    
target_products_in_model = [i for i in target_products if i in model.metabolites]    
excluded_compounds = set(target_products) - set(target_products_in_model)

reachability_tester = reachability.ReachabilityTester(model, target_products_in_model, flux_bound=1000)

df_reachability = reachability_tester.single_gene_reachability()

df_reachability_square = reachability.to_square_form(df_reachability)

f = reachability_plots.plot_clustermap(df_reachability_square)

f.show()

Initializing Reachability Tester using
Model: Recon3DModel_3_01
- Nº of reactions: 10600
- Nº of metabolites: 5835
- Nº of target products: 77
Warning the following target products cannot be reached
glygn2_c xolest_hs_c
Removing compounds form the list of target products
Updating LP models


In [37]:
um_finder = UmFinder(model)
trimed_model = model.copy()
print("===========================")
show(trimed_model)

print("===========================")
gap_metabolites = [trimed_model.metabolites.get_by_id(m) for m in um_finder.gap_metabolites]
blocked_reactions = [trimed_model.reactions.get_by_id(r) for r in um_finder.blocked_reactions]

trimed_model.remove_metabolites(gap_metabolites)
trimed_model.remove_reactions(blocked_reactions)

trimed_model.optimize()

sbml_fname_out = "Recon3DModel_301_no_isoforms_gene_symbols_rpmi_trimed.xml"
write_sbml_model(trimed_model, sbml_fname_out)

Initializing UmFinder Builder using
Model: Recon3DModel_3_01
- Nº of reactions: 10600
- Nº of metabolites: 5835

Checking network consistency (may take some minutes)
Finding blocked reaction using method: fva

- Nº of blocked reactions: 1656
- Nº of gap metabolites: 967
- Nº of unconnected modules: 427
- N of reactions in the biggest unconnected module: 84
- N of metabolites in the biggest unconnected module: 66


NameError: name 'show' is not defined